## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd

import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns=['EIN', 'NAME'])

In [3]:
# Determine the number of unique values in each column.
unique_value_counts = application_df.nunique()
print(unique_value_counts)

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64


In [4]:
# Look at APPLICATION_TYPE value counts for binning
application_type_counts = application_df['APPLICATION_TYPE'].value_counts()
print(application_type_counts)

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64


In [5]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
cutoff_value = 500

# Create a list of application types to be replaced with "Other"
application_types_to_replace = application_type_counts[application_type_counts < cutoff_value].index.tolist()

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Look at CLASSIFICATION value counts for binning
classification_counts = application_df['CLASSIFICATION'].value_counts()
print(classification_counts)

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64


In [7]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classification_counts_greater_than_1 = classification_counts[classification_counts > 1]
print(classification_counts_greater_than_1)

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64


In [8]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
cutoff_value = 1000

# Create a list of CLASSIFICATION values to be replaced with "Other"
classifications_to_replace = classification_counts[classification_counts < cutoff_value].index.tolist()

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [12]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df = pd.get_dummies(application_df)

In [13]:
# Split our preprocessed data into our features and target arrays
# Define the target variable
y = application_df['IS_SUCCESSFUL']

# Define the features by dropping the target variable
X = application_df.drop(columns=['IS_SUCCESSFUL'])

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Check the shapes of the resulting datasets
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

X_train shape: (27439, 43)
X_test shape: (6860, 43)
y_train shape: (27439,)
y_test shape: (6860,)


In [14]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model #2

In [15]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=2, activation="tanh", input_dim=43))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=1, activation="tanh"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 2)                 88        
                                                                 
 dense_1 (Dense)             (None, 1)                 3         
                                                                 
 dense_2 (Dense)             (None, 1)                 2         
                                                                 
Total params: 93 (372.00 Byte)
Trainable params: 93 (372.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [16]:
# Compile the model
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics='accuracy')

In [17]:
# Train the model
nn.fit(X_train_scaled, y_train, epochs=100, verbose=1)

Epoch 1/100
858/858 [==============================] - 6s 5ms/step - loss: 0.6361 - accuracy: 0.6763
Epoch 2/100
858/858 [==============================] - 4s 4ms/step - loss: 0.5817 - accuracy: 0.7302
Epoch 3/100
858/858 [==============================] - 3s 4ms/step - loss: 0.5753 - accuracy: 0.7319
Epoch 4/100
858/858 [==============================] - 5s 6ms/step - loss: 0.5737 - accuracy: 0.7316
Epoch 5/100
858/858 [==============================] - 2s 2ms/step - loss: 0.5733 - accuracy: 0.7317
Epoch 6/100
858/858 [==============================] - 2s 2ms/step - loss: 0.5729 - accuracy: 0.7316
Epoch 7/100
858/858 [==============================] - 2s 2ms/step - loss: 0.5726 - accuracy: 0.7313
Epoch 8/100
858/858 [==============================] - 2s 2ms/step - loss: 0.5724 - accuracy: 0.7320
Epoch 9/100
858/858 [==============================] - 2s 2ms/step - loss: 0.5720 - accuracy: 0.7315
Epoch 10/100
858/858 [==============================] - 2s 2ms/step - loss: 0.5715 - accura

In [18]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

215/215 - 0s - loss: 0.5646 - accuracy: 0.7249 - 409ms/epoch - 2ms/step
Loss: 0.5645699501037598, Accuracy: 0.7249271273612976


In [19]:
# Export our model to HDF5 file
# Define the file path for saving the model
model_filename = "model2.h5"

# Save the model to an HDF5 file
nn.save(model_filename)

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


## Compile, Train and Evaluate the Model #3

In [32]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

nn_3 = tf.keras.models.Sequential()

# First hidden layer
nn_3.add(tf.keras.layers.Dense(units=4, activation="tanh", input_dim=43))

# Second hidden layer
nn_3.add(tf.keras.layers.Dense(units=2, activation="tanh"))

# Output layer
nn_3.add(tf.keras.layers.Dense(units=1, activation="tanh"))

# Check the structure of the model
nn_3.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 4)                 176       
                                                                 
 dense_13 (Dense)            (None, 2)                 10        
                                                                 
 dense_14 (Dense)            (None, 1)                 3         
                                                                 
Total params: 189 (756.00 Byte)
Trainable params: 189 (756.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [33]:
# Compile the model
nn_3.compile(loss='binary_crossentropy', optimizer='adam', metrics='accuracy')

In [34]:
# Train the model
nn_3.fit(X_train_scaled, y_train, epochs=100, verbose=1)

Epoch 1/100
858/858 [==============================] - 3s 3ms/step - loss: 1.8101 - accuracy: 0.6495
Epoch 2/100
858/858 [==============================] - 3s 3ms/step - loss: 0.9673 - accuracy: 0.6907
Epoch 3/100
858/858 [==============================] - 2s 3ms/step - loss: 0.9373 - accuracy: 0.7101
Epoch 4/100
858/858 [==============================] - 2s 2ms/step - loss: 0.9198 - accuracy: 0.7165
Epoch 5/100
858/858 [==============================] - 2s 2ms/step - loss: 0.8040 - accuracy: 0.7176
Epoch 6/100
858/858 [==============================] - 2s 2ms/step - loss: 0.7605 - accuracy: 0.7182
Epoch 7/100
858/858 [==============================] - 2s 2ms/step - loss: 0.6100 - accuracy: 0.7192
Epoch 8/100
858/858 [==============================] - 2s 3ms/step - loss: 0.5856 - accuracy: 0.7203
Epoch 9/100
858/858 [==============================] - 2s 2ms/step - loss: 0.5748 - accuracy: 0.7208
Epoch 10/100
858/858 [==============================] - 2s 2ms/step - loss: 0.5696 - accura

In [35]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_3.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

215/215 - 0s - loss: 0.5587 - accuracy: 0.7310 - 396ms/epoch - 2ms/step
Loss: 0.5586906671524048, Accuracy: 0.7310495376586914


In [ ]:
# Export our model to HDF5 file
# Define the file path for saving the model
model_filename = "model3.h5"

# Save the model to an HDF5 file
nn_3.save(model_filename)

## Compile, Train and Evaluate the Model #4

In [53]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

nn_4 = tf.keras.models.Sequential()

# First hidden layer
nn_4.add(tf.keras.layers.Dense(units=4, activation="tanh", input_dim=43))

# Second hidden layer
nn_4.add(tf.keras.layers.Dense(units=3, activation="tanh"))

# Third hidden layer
nn_4.add(tf.keras.layers.Dense(units=2, activation="tanh"))

# Output layer
nn_4.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_4.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_31 (Dense)            (None, 4)                 176       
                                                                 
 dense_32 (Dense)            (None, 3)                 15        
                                                                 
 dense_33 (Dense)            (None, 2)                 8         
                                                                 
 dense_34 (Dense)            (None, 1)                 3         
                                                                 
Total params: 202 (808.00 Byte)
Trainable params: 202 (808.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [54]:
# Compile the model
nn_4.compile(loss='binary_crossentropy', optimizer='adam', metrics='accuracy')

In [57]:
# Train the model
nn_4.fit(X_train_scaled, y_train, epochs=200, verbose=1)

Epoch 1/200
858/858 [==============================] - 3s 3ms/step - loss: 0.5493 - accuracy: 0.7315
Epoch 2/200
858/858 [==============================] - 2s 2ms/step - loss: 0.5491 - accuracy: 0.7313
Epoch 3/200
858/858 [==============================] - 2s 2ms/step - loss: 0.5494 - accuracy: 0.7310
Epoch 4/200
858/858 [==============================] - 2s 2ms/step - loss: 0.5490 - accuracy: 0.7311
Epoch 5/200
858/858 [==============================] - 2s 3ms/step - loss: 0.5491 - accuracy: 0.7311
Epoch 6/200
858/858 [==============================] - 2s 2ms/step - loss: 0.5492 - accuracy: 0.7313
Epoch 7/200
858/858 [==============================] - 2s 2ms/step - loss: 0.5492 - accuracy: 0.7311
Epoch 8/200
858/858 [==============================] - 2s 2ms/step - loss: 0.5490 - accuracy: 0.7311
Epoch 9/200
858/858 [==============================] - 2s 2ms/step - loss: 0.5491 - accuracy: 0.7314
Epoch 10/200
858/858 [==============================] - 2s 2ms/step - loss: 0.5492 - accura

In [56]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_4.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

215/215 - 0s - loss: 0.5606 - accuracy: 0.7251 - 422ms/epoch - 2ms/step
Loss: 0.5606348514556885, Accuracy: 0.7250728607177734


In [ ]:
# Export our model to HDF5 file
# Define the file path for saving the model
model_filename = "model4.h5"

# Save the model to an HDF5 file
nn_4.save(model_filename)